# Automated Reconstruction of Colombian Public Procurement Contracting Chain

In [14]:
import pandas as pd

# Files with functions
import data_extraction_functions as extract
import data_cleaning_functions as clean
import string_similarity_functions as ss

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/melissamontes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/melissamontes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Code sample
This notebook shows the execution of the construction of the contracting chain. For this we will take one contract from the public entity 'INSTITUTO NACIONAL DE VIAS (INVIAS)' issued to the 'MUNICIPALITY OF PALMIRA', and we will show the procedure to chain the corresponding contracts. The contract has already been preprocessed, so this notebook will only show the chaining through similarity in the pipeline. 

In [5]:
df_entity = pd.read_csv('test_entity_df.csv')
df_entity.head(2)

,Unnamed: 0,uid,anno_cargue_secop,anno_firma_del_contrato,nivel_entidad,orden_entidad,nombre_de_la_entidad,nit_de_la_entidad,c_digo_de_la_entidad,id_tipo_de_proceso,...,marcacion_adiciones,posicion_rubro,nombre_rubro,valor_rubro,sexo_replegal_entidad,pilar_acuerdo_paz,punto_acuerdo_paz,municipio_entidad,departamento_entidad,nom_raz_soc_stand
0,537,14-12-2827756-2648234,2014,2013.0,NACIONAL,NACIONAL CENTRALIZADO,INSTITUTO NACIONAL DE VÍAS (INVIAS),800215807,124002002,12,...,No,No Definido,No Definido,0,ND,No Definido,No Definido,Bogotá D.C.,Bogotá D.C.,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE PALMIRA
1,986,14-12-2470229-2320742,2014,2013.0,NACIONAL,NACIONAL CENTRALIZADO,INSTITUTO NACIONAL DE VÍAS (INVIAS),800215807,124002002,12,...,No,No Definido,No Definido,0,ND,No Definido,No Definido,Bogotá D.C.,Bogotá D.C.,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE PALMIRA


## Construction of the contracting chain
We perform an algorithm to construct the contracting chain for each of the contracts that the public entity issued. The algorithm will have to search in two dataframes:

- Dataframe 1: Contracts issued by the public entity to the municipality/department
- Dataframe 2: Contracts issued by the municipality/department to a private contractor

The algorithm will:

1. Select a contract issued by the public entity to the municipality/department (Dataframe 1)
2. Look all the contract issued by the municipality/department to a private contractor (Dataframe 2), and search for the contracts with the highest similarity with the description of the 1st contract (1).
3. The contracts with a similarity measure higher than 0.8 will be chained.

The resulting dataframe will have:
- A row for each chained contracts, with all the information available in SECOP for both contracts


We will show the steps listed above with a contract from the dataframe obtained above. This procedure will be repeated for all the contracts issued to each municipality in the public entity database.

In [11]:
# Parameters
chain_cont = 0
threshold = 0.8
chain_df = pd.DataFrame()

# Small sample of municipalities
list_mun = ['VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE PALMIRA']
n_contracts = 2

entity_contracts = df_entity[:1]
entity_contracts

,Unnamed: 0,uid,anno_cargue_secop,anno_firma_del_contrato,nivel_entidad,orden_entidad,nombre_de_la_entidad,nit_de_la_entidad,c_digo_de_la_entidad,id_tipo_de_proceso,...,marcacion_adiciones,posicion_rubro,nombre_rubro,valor_rubro,sexo_replegal_entidad,pilar_acuerdo_paz,punto_acuerdo_paz,municipio_entidad,departamento_entidad,nom_raz_soc_stand
0,537,14-12-2827756-2648234,2014,2013.0,NACIONAL,NACIONAL CENTRALIZADO,INSTITUTO NACIONAL DE VÍAS (INVIAS),800215807,124002002,12,...,No,No Definido,No Definido,0,ND,No Definido,No Definido,Bogotá D.C.,Bogotá D.C.,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE PALMIRA


### Obtaining contracts issued by the municipality

In the previous step we have a contract issued by the public entity to the MUNICIPALITY OF PALMIRA. Now we get all the contracts issued BY the MUNICIPALITY OF PALMIRA to construct the second part of the chain. 

In [18]:
mun_name = list_mun[0]

# Subsets public entity df to contracts issued for the mun/dept.
entity_contracts_mun = entity_contracts.loc[entity_contracts['nom_raz_soc_stand'] == mun_name]

mun_contracts = extract.extract_mun_contracts(mun_name)
# If there are no contracts for the mun/dept. in SECOP continue
if mun_contracts.empty:
    print('No contracts')

mun_contracts = clean.df_cleaning(mun_contracts)
# If there are no contracts for the mun/dept. in the states allowed, continue
# States = 'Liquidado', 'Terminado Sin Liquidar', 'Celebrado', 'Adjudicado', 'Convocado'
if mun_contracts.empty:
    print('No contracts with allowed states')
    
mun_contracts.head(2)

,uid,anno_cargue_secop,anno_firma_del_contrato,nivel_entidad,orden_entidad,nombre_de_la_entidad,nit_de_la_entidad,c_digo_de_la_entidad,id_tipo_de_proceso,tipo_de_proceso,...,espostconflicto,marcacion_adiciones,posicion_rubro,nombre_rubro,valor_rubro,sexo_replegal_entidad,pilar_acuerdo_paz,punto_acuerdo_paz,municipio_entidad,departamento_entidad
0,18-12-8271102-7523971,2018,2018.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 1,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE PALMIRA,800094449-8,276520011,12,Contratación Directa (Ley 1150 de 2007),...,No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,Palmira,Valle del Cauca
1,19-12-10086535-9208925,2019,2019.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 1,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE PALMIRA,800094449-8,276520011,12,Contratación Directa (Ley 1150 de 2007),...,No Definido,No,No Definido,No Definido,0,Hombre,No Definido,No Definido,Palmira,Valle del Cauca


### Similarity measure
Now that we have all the contracts issued by the MUNICIPALITY OF PALMIRA, we compare them to the first contract and assign a similarity measure for all the comparisons. 

Before comparing the contracts, we make sure they are issued on or after the year of the first contract, and we clean the 'detail of the contract' strings. 

The similarity measure is performed by the following steps:

1. Transforms 'description of the contract' for both databases with tf-idf. We tokenize by 3-grams to reduce noise from typing errors.
2. Gets similarity scores with cosine similarity.
3. Converts similarity matrix (sparse) to a readable df

In [25]:
# Only evaluate mun/dept contracts issued on or after year of the entity contract
mun_contracts_filter = mun_contracts[
    (mun_contracts.anno_firma_del_contrato >= entity_row['anno_firma_del_contrato']) |
    (mun_contracts['anno_firma_del_contrato'].isnull())]
mun_contracts_filter = mun_contracts_filter.reset_index(drop=True)

# Create list with description of each contract to evaluate
mun_description = mun_contracts_filter['detalle_del_objeto_a_contratar']
mun_description_list = list(map(str, list(map(clean.standarize_obj, mun_description))))
mun_description_list = pd.Series(mun_description_list)
entity_description_list = pd.Series(clean.standarize_obj(entity_row['detalle_del_objeto_a_contratar']))

# Paste descriptions mun/dept. and entity. Last row = entity contract
description_list = list(mun_description_list) + list(entity_description_list)

# String similarity algorithm -----
# 1. Transforms strings with tf-idf algorithm to a numeric matrix
tf_idf_matrix = ss.tf_idf(description_list)
# 2. Gets similarity scores in a sparse matrix
matches_sparse = ss.awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), n_contracts)
# 3. Converts similarity matrix to a readable df
matches_df = ss.get_matches_df(matches_sparse, description_list)


matches_df_mun = matches_df.loc[matches_df['pos_left'] == (len(description_list)-1)]
matches_df_mun.head(2)

,left_side,right_side,similarity,pos_left,pos_right
10986,MEJORAMIENTO MANTENIMIENTO CONSERVACION VIA BU...,MEJORAMIENTO MANTENIMIENTO CONSERVACION VIA BU...,1.000000,5493,5493
10987,MEJORAMIENTO MANTENIMIENTO CONSERVACION VIA BU...,MEJORAMIENTO MANTENIMIENTO CONSERVACION BUITRE...,0.920339,5493,1579


### Chain building with similarity measure
Now that we have all the similarity scores for the contracts, we chain the information of both contracts, having as IDs the position of each contract (pos_left and pos_right). We establish that a contract with a similitude score above 0.8 belongs to the contracting chain.

In this case we also chain the contracts with an score below 0.8 for validation purposes.

In [31]:
 # Chain construction ----
# Pastes complete info of the contracts with high similitude
for index_chain, row_chain in matches_df_mun.iterrows():
    if row_chain['pos_left'] == row_chain['pos_right']: continue
    score = row_chain['similarity']
    # Joining info
    chain_entity = entity_row.to_frame().T
    if len(mun_contracts_filter) == row_chain['pos_right']: continue
    chain_mun = mun_contracts_filter.iloc[row_chain['pos_right']].to_frame().T
    chain_entity.index = [chain_cont]
    chain_mun.index = [chain_cont]
    chain_mun.columns = [str(col) + '_mun' for col in chain_mun.columns]
    chain_result = pd.concat([chain_entity, chain_mun], axis=1, join='inner', sort=True)
    chain_df = chain_df.append(chain_result, sort=False)
    chain_df.at[chain_df.index[chain_cont], 'score'] = score
    if score > threshold:
        chain_df.at[chain_df.index[chain_cont], 'valid'] = True
    else:
        chain_df.at[chain_df.index[chain_cont], 'valid'] = False
    chain_cont = chain_cont + 1
    
chain_df.head(5)    

,Unnamed: 0,uid,anno_cargue_secop,anno_firma_del_contrato,nivel_entidad,orden_entidad,nombre_de_la_entidad,nit_de_la_entidad,c_digo_de_la_entidad,id_tipo_de_proceso,...,posicion_rubro_mun,nombre_rubro_mun,valor_rubro_mun,sexo_replegal_entidad_mun,pilar_acuerdo_paz_mun,punto_acuerdo_paz_mun,municipio_entidad_mun,departamento_entidad_mun,score,valid
0,537,14-12-2827756-2648234,2014,2013,NACIONAL,NACIONAL CENTRALIZADO,INSTITUTO NACIONAL DE VÍAS (INVIAS),800215807,124002002,12,...,No Definido,No Definido,0,ND,No Definido,No Definido,Palmira,Valle del Cauca,0.920339,True
1,537,14-12-2827756-2648234,2014,2013,NACIONAL,NACIONAL CENTRALIZADO,INSTITUTO NACIONAL DE VÍAS (INVIAS),800215807,124002002,12,...,No Definido,No Definido,0,ND,No Definido,No Definido,Palmira,Valle del Cauca,0.920339,True
2,537,14-12-2827756-2648234,2014,2013,NACIONAL,NACIONAL CENTRALIZADO,INSTITUTO NACIONAL DE VÍAS (INVIAS),800215807,124002002,12,...,No Definido,No Definido,0,ND,No Definido,No Definido,Palmira,Valle del Cauca,0.920339,True
3,537,14-12-2827756-2648234,2014,2013,NACIONAL,NACIONAL CENTRALIZADO,INSTITUTO NACIONAL DE VÍAS (INVIAS),800215807,124002002,12,...,No Definido,No Definido,0,ND,No Definido,No Definido,Palmira,Valle del Cauca,0.920339,True


## Final thoughts

The previous script showed the string similarity procedure to construct the contracting chain for an specific municipality. This procedure will be repeated for all contracts in municipalities that issued a contract with any public entity.